In [22]:
import os
import random
import logging
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf

from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utils.transformation import *
from utils.models import *

VAR_SEED = 42
VAR_TESTSET_SIZE = 0.20
VAR_DIR_DATA_CLEAN = '../data/cleaning'

random.seed(VAR_SEED)
np.random.seed(VAR_SEED)
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
tf.get_logger().setLevel(logging.ERROR)

df_items = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/catalogo.csv", sep=",", encoding="latin1")
mf_dataset = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/mf_dataset.csv", sep=",", encoding="latin1")
dataset = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/dataset.csv", sep=",", encoding="latin1")
prueba_diagnostico = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/prueba_diagnostico.csv", sep=",", encoding="latin1") 

In [23]:
# catalogo.drop(labels=['nombre', 'hito', 'skill', 'knowledge', 'complexity', 'complexity12'], axis=1).head()
# mf_dataset.head()
# dataset[['id_estudiante', 'programa', 'exitosos', 'fallidos']].head()
# prueba_diagnostico[['id_estudiante', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'op1', 'op2', 'op3', 'op4', 'op5', 'op6', 'score', 'score_a', 'score_p', 'score_d', 'score_s']].head()

df_usuarios = dataset[['id_estudiante','exitosos', 'fallidos']].merge(mf_dataset, on='id_estudiante', how='left')
df_usuarios

,id_estudiante,exitosos,fallidos,e0,e1,e2,e3,e4,e5,e6,...,e43,e44,e45,e46,e47,e48,e49,e50,e51,e52
0,0,15,24,1,1,0,1,1,0,1,...,0,1,0,0,0,0,0,0,0,0
1,1,10,6,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,11,20,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,7,17,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,13,18,1,1,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,1301,11,5,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
762,1302,9,7,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
763,1303,11,20,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
764,1304,16,15,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df_usuarios.columns

Index(['id_estudiante', 'exitosos', 'fallidos', 'e0', 'e1', 'e2', 'e3', 'e4',
       'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12', 'e13', 'e14', 'e15',
       'e16', 'e17', 'e18', 'e19', 'e20', 'e21', 'e22', 'e23', 'e24', 'e25',
       'e26', 'e27', 'e28', 'e29', 'e30', 'e31', 'e32', 'e33', 'e34', 'e35',
       'e36', 'e37', 'e38', 'e39', 'e40', 'e41', 'e42', 'e43', 'e44', 'e45',
       'e46', 'e47', 'e48', 'e49', 'e50', 'e51', 'e52'],
      dtype='object')

In [25]:
df_items.columns

Index(['id_ejercicio', 'nombre', 'h1', 'h2', 'h3', 'h4', 's1', 's2', 's3',
       's4', 'k1', 'k2', 'k3', 'k4', 'hito', 'skill', 'knowledge',
       'complexity', 'complexity12'],
      dtype='object')

In [30]:
# Crear un DataFrame de interacciones
interacciones = []
for _, usuario in df_usuarios.iterrows():
    for i in range(3):  # Suponiendo 3 ejercicios para simplificar
        if usuario[f'e{i}'] == 1:
            interacciones.append((usuario['id_estudiante'], df_items.iloc[i]['id_ejercicio'], 1))
        else:
            interacciones.append((usuario['id_estudiante'], df_items.iloc[i]['id_ejercicio'], 0))

df_interacciones = pd.DataFrame(interacciones, columns=['id_estudiante', 'id_ejercicio', 'interactuo'])

In [31]:
df_interacciones

,id_estudiante,id_ejercicio,interactuo
0,0,0,1
1,0,1,1
2,0,2,0
3,1,0,1
4,1,1,0
...,...,...,...
2293,1304,1,0
2294,1304,2,0
2295,1305,0,0
2296,1305,1,0


In [32]:
# Convertir los IDs a arrays
X_user = df_interacciones['id_estudiante'].values
X_item = df_interacciones['id_ejercicio'].values
y = df_interacciones['interactuo'].values  # Etiquetas

# Asegurarnos de que sean enteros
X_user = X_user.astype(np.int32)
X_item = X_item.astype(np.int32)
y = y.astype(np.float32)  # Para la pérdida binaria


In [33]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

# Parámetros
num_users = df_usuarios['id_estudiante'].nunique() + 1  # +1 para incluir el índice 0
num_items = df_items['id_ejercicio'].nunique() + 1  # +1 para incluir el índice 0
embedding_size = 64  # Tamaño del embedding

# Torre de Usuarios
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size)(user_input)
user_vec = Flatten()(user_embedding)

# Torre de Ítems
item_input = Input(shape=(1,), name='item_input')
item_embedding = Embedding(input_dim=num_items, output_dim=embedding_size)(item_input)
item_vec = Flatten()(item_embedding)

# Concatenar los vectores de usuario e ítem
merged = Concatenate()([user_vec, item_vec])
output = Dense(1, activation='sigmoid')(merged)  # Predicción de interacción

# Crear el modelo
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 64)     │     49,088 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 64)     │      3,456 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 64)        │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 64)        │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 128)       │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        129 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 52,673 (205.75 KB)

 Trainable params: 52,673 (205.75 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Entrenar el modelo
model.fit([X_user, X_item], y, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10


2024-10-27 16:38:57.231201: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: indices[0,0] = 965 is not in [0, 767)
	 [[{{function_node __inference_one_step_on_data_1293}}{{node functional_1_1/embedding_2_1/GatherV2}}]]


InvalidArgumentError: Graph execution error:

Detected at node functional_1_1/embedding_2_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/edgar-ramxs/sistema-de-recomendacion-unab/.sistema-de-recomendacion-unab-env/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/edgar-ramxs/sistema-de-recomendacion-unab/.sistema-de-recomendacion-unab-env/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/edgar-ramxs/sistema-de-recomendacion-unab/.sistema-de-recomendacion-unab-env/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/edgar-ramxs/sistema-de-recomendacion-unab/.sistema-de-recomendacion-unab-env/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_

In [ ]:
def recommend_items(user_id, model, num_recommendations=5):
    # Crear un array de todos los ejercicios
    all_items = df_items['id_ejercicio'].unique()
    user_items = [user_id] * len(all_items)
    
    # Predecir interacciones
    preds = model.predict([user_items, all_items])
    
    # Obtener las recomendaciones
    recommended_indices = preds.flatten().argsort()[-num_recommendations:][::-1]
    recommended_items = all_items[recommended_indices]
    
    return recommended_items

# Ejemplo de recomendaciones para un usuario
recommended_items = recommend_items(user_id=1, model=model)
print(recommended_items)
